# Day 25: Clock Signal

## Part One

You open the door and find yourself on the roof. The city sprawls away from you for miles and miles.

There's not much time now - it's already Christmas, but you're nowhere near the North Pole, much too far to deliver these stars to the sleigh in time.

However, maybe the huge antenna up here can offer a solution. After all, the sleigh doesn't need the stars, exactly; it needs the timing data they provide, and you happen to have a massive signal generator right here.

You connect the stars you have to your prototype computer, connect that to the antenna, and begin the transmission.

Nothing happens.

You call the service number printed on the side of the antenna and quickly explain the situation. "I'm not sure what kind of equipment you have connected over there," he says, "but you need a clock signal." You try to explain that this is a signal for a clock.

"No, no, a clock signal - timing information so the antenna computer knows how to read the data you're sending it. An endless, alternating pattern of `0`, `1`, `0`, `1`, `0`, `1`, `0`, `1`, `0`, `1`...." He trails off.

You ask if the antenna can handle a clock signal at the frequency you would need to use for the data from the stars. "There's no way it can! The only antenna we've installed capable of that is on top of a top-secret Easter Bunny installation, and you're definitely not-" You hang up the phone.

You've extracted the antenna's clock signal generation assembunny code (your puzzle input); it looks mostly compatible with code you worked on just recently.

This antenna code, being a signal generator, uses one extra instruction:

* `out x` transmits x (either an integer or the value of a register) as the next value for the clock signal.

The code takes a value (via register a) that describes the signal to generate, but you're not sure how it's used. You'll have to find the input to produce the right signal through experimentation.

What is the lowest positive integer that can be used to initialize register `a` and cause the code to output a clock signal of `0`, `1`, `0`, `1`... repeating forever?

---

In [1]:
# Initialise
inputs = [i[:-1].split(' ') for i in open('Day25.in').readlines()]
# inputs = [i.split(' ') for i in test.split('\n')]

register = {
    char: 0 for char in ('a', 'b', 'c', 'd')
}

register['a'] = 7

In [ ]:
# Tweak the code to include tgl
def assembunny(reg, cmds, cursor=0):
    """
    Takes a register and commands then parses and runs the commands. It assumes cursor 
    starts at 0 but may be set as a option.
    
    :returns: None
    """
    # This function will parse a command (from cmds) at the given cursor position
    def parse(cursor, out=[]):
        cmd, x, y, *_ = cmds[cursor] + [None]
        if cmd == 'cpy':
            if x in reg:
                reg[y] = reg[x]
            else:
                reg[y] = int(x)
        elif cmd == 'jnz':
            # Put -1 in here because we'll increment the cursor at the end
            y = reg[y] if y in reg else int(y)
#             if delta:= optimise_jnz(reg, cmds[cursor + y:cursor], x):
#                 for char in reg:
#                     reg[char] += delta[char]
            if x in reg:
                cursor += y - 1 if reg[x] else 0
            else:
                cursor += y - 1 if int(x) else 0
        elif cmd == 'out':
            out.append(reg[x])
            print(', '.join(out))
        elif cmd == 'tgl':
            x = reg[x] if x in reg else int(x)
            tgl, a, b, *_ = cmds[cursor + x] + [None]
            if b is None:
                cmds[cursor + x] = [('dec' if tgl == 'inc' else 'inc'), a]
            else:
                cmds[cursor + x] = [('cpy' if tgl == 'jnz' else 'jnz'), a, b]
        else:
            reg[x] += 1 if cmd == 'inc' else -1
        return cursor + 1
    
    cursors = []
    high_cursor = 0
    out = []
    
    # Run cmds until cursor exceeds len(cmds)
    while cursor < len(cmds):
#         cursors.append(f"{cursor}: {str(cmds[cursor]).ljust(18)} {reg}")
#         clear_output(wait=True)
#         print(f'highest cursor value: {high_cursor}\n' + '\n'.join(cursors))
#         time.sleep(0.1)
        try:
            cursor = parse(cursor, out)
        except:
#             print("WOOPS")
            cursor += 1
        
#         if not reg['a'] % 10_000:
#             clear_output(wait=True)
#             print(reg)
#         if len(cursors) > 10:
#             cursors = cursors[1:]
#         if cursor > high_cursor:
#             high_cursor = cursor

def optimise_jnz(reg, cmds, jnz_char):
    """
    Given repeating arguments prior to a jnz optimize into one step if possible, else return None
    
    If possible, return delta as {a: delta_a, ..., d: delta_d}
    """
    if jnz_char not in ('a', 'b', 'c', 'd'):
        return None
    
    delta = {
        char: 0 for char in ('a', 'b', 'c', 'd')
    }
    
    if not [cmd for cmd in cmds if cmd[0] in ('tgl', 'jnz', 'cpy')]:
        for cmd in cmds:
            if cmd[0] == 'inc':
                delta[cmd[1]] += 1
            else:
                delta[cmd[1]] -= 1
    
        jnz_val = delta[jnz_char]
        
        return {
            char: int(delta[char]*reg[jnz_char]/(-jnz_val)) for char in delta
        }
        

# Run inputs through assumbunny
assembunny(register, inputs)

# Solution
print(f"The value {register['a']:,} is left in register a.")